# Imports

In [177]:
import textract # extract the text from the image
import numpy as np # use to dealing with arrays and matrices
import cv2 # dealing with videos and images
import os # dealing with operating system
import datetime # dealing with data and time
import openpyxl # used to dealing with excel sheets

# Filter Function

In [178]:
# This function is used to filter the text to extract the code from the card
def filter(list_text):    
    needed_text = 'Code : '    
    for i in list_text:
        # Check if the element is numeric
        if i.isnumeric():
            needed_text+=i +' '
    return needed_text 

# Read the students excel file

In [183]:
# load the excel sheet
workbook = openpyxl.load_workbook('Students.xlsx')
# select the sheet from the file
sheet = workbook['ورقة1']  

# initialize dict to store students names and IDs
attend_list ={}
# loop on the sheet rows to extract the students name and ID
for student_name, student_id in sheet.iter_rows(min_row=1, values_only=True):
    attend_list[f'Code : {student_id} '] = student_name

attend_list

{'Code : 205096 ': 'Ahmed Khattab',
 'Code : 205004 ': 'Omar Mohamed',
 'Code : 205036 ': 'Mohamed Fayez',
 'Code : 205094 ': 'Abdelnasser'}

# Incialize the important variables

In [181]:
# the list that contains the recorded studen
recorded_ID = []

# get the date of the day
current_data = datetime.datetime.now().date()

# Main program 

In [182]:
# Create a workbook - represents an Excel file
workbook = openpyxl.Workbook()
# activate the sheet to allow write on it
sheet = workbook.active

# video capture
cap = cv2.VideoCapture(0)

# main loop of the program
while cap.isOpened():
    # get the time in hours and minuts and seconds
    current_time = datetime.datetime.now().hour
    current_min = datetime.datetime.now().minute
    current_sec = datetime.datetime.now().second
    
    # capture the frames of the video
    ret, frame = cap.read()
    if ret:                
        # Apply processing to frames
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Apply GaussianBlur to the frames
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        
        # THRESH_OTSU automatically determine the optimal threshold based on the image histogram.
        _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) # converts the grayscale image into a binary image
        
        
        # useful for general noise removal and smoothing effects.
        # kernel = np.array([[1, 0, 1],
        #                    [0, 1, 0],
        #                    [1, 0, 1]], dtype=np.uint8)
        
        
        # useful for enhancing and thickening text or line-like structures.
        kernel = np.array([[0, 1, 0],
                           [1, 1, 1],
                           [0, 1, 0]], dtype=np.uint8)
        
        # Apply dilation and erosion on the binary image
        dilated = cv2.dilate(thresh, kernel, iterations=1)
        eroded = cv2.erode(dilated, kernel, iterations=1)
        
        
        image_path = "temp.png"  # Temporary file path
        cv2.imwrite(image_path, eroded)  # Save the image frame
        
        # Process the saved image file using textract
        text = textract.process(image_path, method='tesseract', language='eng') # Tesseract is commonly used for extracting text from images.
       
        # function returns the extracted text as bytes so we need to use (decode) to convert it to string 
        text = text.decode('utf-8')
        
        list_text = text.split() # tokenize the text into words.
        
        # filtering the text to extract the part of interest.
        needed_text = filter(list_text)
                
        # check if the ID is in our attendance list of students.
        if needed_text in attend_list:
            
            #check if the student is already attend
            if needed_text not in recorded_ID:
                
                # add the student to recorded ID list.
                recorded_ID.append(needed_text)
                
                # print student name - ID is recorded
                print(f'{attend_list[needed_text]}: {needed_text} is recorded')
    
                # row is an list that contain the name and the ID
                row = [attend_list[needed_text], needed_text , f'Date : {current_data}' , f'Hours : {current_time}' , f'Minuts {current_min}', f'Seconds {current_sec}']
                # add this list to the attendance sheet
                sheet.append(row)
           
            else: 
                # if the student is already recorded
                print(f'{attend_list[needed_text]}: {needed_text} is already recorded')
        
        
        # Save the Excel sheet as attendance.xlsx
        workbook.save('attendance.xlsx')  
        
        # display the video to students
        cv2.imshow("card", eroded)
        
        # check if the user is press q
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break
    # if the video frams stop readed
    else:
        break

# release the capture
cap.release()
# destroy the window after program is terminate
cv2.destroyAllWindows()
# remove the temporary image
os.remove('temp.png')


Ahmed Khattab: Code : 205096  is recorded
Ahmed Khattab: Code : 205096  is already recorded
Omar Mohamed: Code : 205004  is recorded
